# Data Reduction

Have too much data to train GP. How can we reduce (this might not be needed for NN??).

In [1]:
import torch
import numpy as np

from sklearn.cluster import DBSCAN
from os.path import join

import sys
sys.path.append('../')
import utils

## Load Data

In [2]:
def combine_data(data_tuple:tuple[torch.tensor]) -> tuple[torch.tensor]:
    '''
    Add Landmass to x data. Return x,y tensors
    Input:
        - data_tuple (tuple[torch.tensor]): tuple of landmass,x,y tensors
    '''
    l,x,y = data_tuple

    # Combine
    l = l.unsqueeze(1)
    x = torch.cat((l, x), 1)
    x = x.contiguous()
    y = y.contiguous()
    return (x,y)

In [3]:
fname = join('/home/squirt/Documents/data/weather_data/', 'all_data.h5')
data = utils.load_hdf5(fname)
stacks = utils.generate_stacks(data)
x, y = combine_data(stacks)
del data, stacks

Can use the mean since this isn't scientific. 

In [5]:
x = torch.mean(x, dim=(3,4))
y = torch.mean(y, dim=(3,4)) 

x = x.view(-1, 71*3)
y = y.view(-1, 70*2)

x = x.numpy()
y = y.numpy()

## Cluster to remove dissimilar x pairs

In [11]:
# Assuming X is your feature matrix
clustering = DBSCAN(eps=0.5, min_samples=5).fit(x)
# Get the cluster labels
labels = clustering.labels_

# Identify unique clusters (excluding noise points labeled as -1)
unique_labels = set(labels) - {-1}

# Select one representative point from each cluster
representatives = []
for label in unique_labels:
    indices = np.where(labels == label)[0]
    representatives.append(x[indices[0]])  # You can choose any point within the cluster

In [14]:
labels

array([-1, -1, -1, ..., -1, -1, -1])

So looks like noise, thus should use all data????